# Question
What are the 5 nearest ports to Singapore's JURONG ISLAND port? (country = 'SG',
port_name = 'JURONG ISLAND').Your answer should include the columns port_name and
distance_in_meters only.

In [ ]:
import psycopg2
import os 

## Connect to the PostgreSQL database

In [ ]:
def get_pg_connection(host, database, user, password):
    pg_conn = psycopg2.connect(host = host, database = database, user = user, password = password)
    return pg_conn
pg_conn = get_pg_connection('localhost', 'Capstone', os.environ['user'], os.environ['password'])
pg_cur = pg_conn.cursor()

## SQL Query 

In [ ]:
sql_query = """
    CREATE TABLE nearest_ports_to_jurong_island AS
    SELECT "Main_port_name", "Wpi_country_code",
           ( 6371000 * ACOS(
        SIN(RADIANS(jurong_latitude_degrees) + (jurong_latitude_minutes / 60.0)) *
        SIN(RADIANS("Latitude_degrees") + ("Latitude_minutes" / 60.0)) +
        COS(RADIANS(jurong_latitude_degrees) + (jurong_latitude_minutes / 60.0)) *
        COS(RADIANS("Latitude_degrees") + ("Latitude_minutes" / 60.0)) *
        COS(RADIANS("Longitude_degrees") + ("Longitude_minutes" / 60.0) - RADIANS(jurong_longitude_degrees) - (jurong_longitude_minutes / 60.0)) 
        )) AS distance_in_meters
    FROM wpi_data, (SELECT "Latitude_degrees" AS jurong_latitude_degrees, "Latitude_minutes" AS jurong_latitude_minutes, "Longitude_degrees" AS jurong_longitude_degrees, "Longitude_minutes" AS jurong_longitude_minutes
                    FROM wpi_data
                    WHERE "Main_port_name" = 'JURONG ISLAND' AND "Wpi_country_code" = 'SG') AS jurong_island
    WHERE "Main_port_name" != 'JURONG ISLAND'
    ORDER BY distance_in_meters
    LIMIT 5
"""
pg_cur.execute(sql_query)

# Commit the changes to the database
pg_conn.commit()

## Close the cursor and connection

In [ ]:
pg_cur.close()
pg_conn.close()